# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. This notebook tests several attributes of the generation process to ensure that the code is sound. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
form = {
    "accountid": "3025284578519",
    "aggression": 0.5,
    "conformity": 0.5,
    "constitution": 0.5,
    "consumes": "organics",
    "effuses": "organic waste,plastics",
    "label": "form",
    "literacy": 0.5,
    "minerals": 0.5,
    "name": "worldgenform",
    "num_moons": 10,
    "num_planets": 4,
    "objid": 300882390249,
    "organics": 0.5,
    "owner": "William Harding",
    "starting_pop": 7,
    "userguid": "8d5b667f-b225-4641-b499-73b77558ff86",
    "username": "William Harding"
}

userguid = form["userguid"]

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()
conf.keys()

dict_keys(['planet_config', 'moon_config', 'star_config', 'resource_config'])

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(form)
system

<system: ordered; 9653477904374; Hisyrezpan>

In [7]:
system.get_data()

{'objid': '9653477904374',
 'name': 'Hisyrezpan',
 'label': 'system',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': 26.611,
 'glon': 14.951,
 'gelat': 4.304}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'Dalpa',
 'class': 'G',
 'objid': '0164935591491',
 'label': 'star',
 'pop_cap': 100,
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'Dalpa',
 'class': 'G',
 'objid': '0164935591491',
 'label': 'star',
 'pop_cap': 100}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Senser',
 'class': 'terrestrial',
 'objid': '7377028193236',
 'label': 'planet',
 'pop_cap': 100,
 'radius': 1.045,
 'mass': 0.227,
 'orbitsDistance': 1.015,
 'orbitsId': '0164935591491',
 'orbitsName': 'Dalpa',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(form["num_planets"]) - 1)
]
planets

KeyError: 'prob'

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(form["num_moons"]))
]
moons

[<moon: rocky; 1599477100132; Dibergbaspol>,
 <moon: rocky; 6964251607202; Barron>,
 <moon: ice; 6673546856014; Bay>,
 <moon: rocky; 3117677310763; Heim>,
 <moon: rocky; 2382952889723; Newrezrang>,
 <moon: rocky; 0196378384557; Quafa>,
 <moon: rocky; 8250077186631; O>,
 <moon: ice; 8773555122015; Tosdrohui>,
 <moon: rocky; 8924559107884; Tal>,
 <moon: rocky; 6907085549719; Bangnue>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: terrestrial; 8814239227413; Ban>

In [14]:
home_planet.orbiting

<star: G; 4645049108359; Senhua>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<system: ordered; 4998076575301; Bersogassha>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,pop_cap,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,4998076575301,Bersogassha,system,ordered,True,16.223,32.395,0.132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4645049108359,Senhua,star,G,NaN,NaN,NaN,NaN,100.0,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,1599477100132,Dibergbaspol,moon,rocky,NaN,NaN,NaN,NaN,100.0,0.180186,8814239227413,Ban,0.6321,0.000074,False,False
3,6964251607202,Barron,moon,rocky,NaN,NaN,NaN,NaN,100.0,0.038119,6357378750821,Kesquetro,0.3051,0.000063,False,False
4,6673546856014,Bay,moon,ice,NaN,NaN,NaN,NaN,100.0,0.014825,6357378750821,Kesquetro,0.4490,0.009125,False,False
5,3117677310763,Heim,moon,rocky,NaN,NaN,NaN,NaN,100.0,0.121268,8814239227413,Ban,0.8060,0.000467,False,False
6,2382952889723,Newrezrang,moon,rocky,NaN,NaN,NaN,NaN,100.0,0.008327,5455391951651,Lespon,0.0790,0.000229,False,False
7,0196378384557,Quafa,moon,rocky,NaN,NaN,NaN,NaN,100.0,0.027566,6357378750821,Kesquetro,0.3051,0.000346,False,False
8,8250077186631,O,moon,rocky,NaN,NaN,NaN,NaN,100.0,0.085600,6357378750821,Kesquetro,0.3390,0.000254,False,False
9,8773555122015,Tosdrohui,moon,ice,NaN,NaN,NaN,NaN,100.0,0.016069,8814239227413,Ban,0.6321,0.012649,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,7581206629313,4645049108359,orbits,0.4810
1,8814239227413,4645049108359,orbits,0.9530
2,5455391951651,4645049108359,orbits,44.1420
3,6357378750821,4645049108359,orbits,0.7090
4,1599477100132,8814239227413,orbits,0.0001
5,6964251607202,6357378750821,orbits,0.0001
6,6673546856014,6357378750821,orbits,0.1440
7,3117677310763,8814239227413,orbits,0.1740
8,2382952889723,5455391951651,orbits,0.0410
9,0196378384557,6357378750821,orbits,0.0001


In [18]:
pd.DataFrame(orbiting_edges).groupby(["node1", "node2"]).count()["label"].mean() == 1

True

In [19]:
pd.DataFrame(orbiting_edges).groupby(["node2"]).count()

,node1,label,orbit_distance
node2,,,
4645049108359,4,4,4
5455391951651,1,1,1
6357378750821,4,4,4
8814239227413,5,5,5


Check the orbiting logic

In [20]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'terrestrial planet orbits a G star',
 'dwarf planet orbits a G star',
 'terrestrial planet orbits a G star',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'ice moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a dwarf planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'ice moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet']

In [21]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,7581206629313,4998076575301,isIn
1,8814239227413,4998076575301,isIn
2,5455391951651,4998076575301,isIn
3,6357378750821,4998076575301,isIn
4,1599477100132,4998076575301,isIn
5,6964251607202,4998076575301,isIn
6,6673546856014,4998076575301,isIn
7,3117677310763,4998076575301,isIn
8,2382952889723,4998076575301,isIn
9,0196378384557,4998076575301,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [25]:
home_planet.config.keys()

dict_keys(['name', 'count', 'prob', 'mass_mean', 'mass_std', 'radius_mean', 'radius_std', 'distance_min', 'distance_max', 'has_atmosphere', 'resources', 'atmosphere'])

In [22]:
home_planet.scan_body()
home_planet.atmosphere

[{'Argon': 0.0},
 {'Carbon Dioxide': 0.095},
 {'Helium': 0.05},
 {'Hydrogen': 0.146},
 {'Methane': 0.003},
 {'Nitrogen': 0.534},
 {'Oxygen': 0.109},
 {'Sodium': 0.063}]

In [23]:
home_planet.get_data()

{'name': 'Elnin',
 'class': 'terrestrial',
 'objid': '7581206629313',
 'label': 'planet',
 'pop_cap': 100,
 'atmosphere': [{'Argon': 0.0},
  {'Carbon Dioxide': 0.095},
  {'Helium': 0.05},
  {'Hydrogen': 0.146},
  {'Methane': 0.003},
  {'Nitrogen': 0.534},
  {'Oxygen': 0.109},
  {'Sodium': 0.063}],
 'radius': 0.777,
 'mass': 0.112,
 'orbitsDistance': 0.481,
 'orbitsId': '4645049108359',
 'orbitsName': 'Senhua',
 'isSupportsLife': False,
 'isPopulated': False}

In [24]:
home_planet.resources

[<resource: 8048320740756; organics>,
 <resource: 9368440053161; common minerals>,
 <resource: 8891873981597; rare minerals>,
 <resource: 6360666896066; water>]

In [26]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,organics,4922300816520,resource,976,976,bilogical material that can be consumed by pops,10.0
1,common minerals,7687415333659,resource,115,115,Iron and other common material used in constru...,NaN
2,rare minerals,3694306733951,resource,50,50,"lithium, silver and other rare minerals used i...",NaN
3,water,9210630543932,resource,10160,10160,"H2O ready to be consumed, either frozen or in ...",NaN


In [27]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,5237515590885,4922300816520,has
1,5237515590885,7687415333659,has
2,5237515590885,3694306733951,has
3,5237515590885,9210630543932,has


In [28]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 976 organics',
 'terrestrial planet has 115 common minerals',
 'terrestrial planet has 50 rare minerals',
 'terrestrial planet has 10160 water']